In [82]:
import os
import re
import pandas as pd

# Path to your folder with .txt files
data_folder = './data'

# Patterns to extract the fields
name_pattern = re.compile(r'COMPANY CONFORMED NAME:\s+(.*)')
cik_pattern = re.compile(r'CENTRAL INDEX KEY:\s+(\d{10})')

# List to hold all extracted records
records = []

count = 0

# Loop through all .txt files in the folder
for filename in os.listdir(data_folder):
    count += 1
    if filename.endswith('.txt'):
        with open(os.path.join(data_folder, filename), 'r', encoding='utf-8', errors='ignore') as file:
            content = file.read()
            name_match = name_pattern.search(content)
            cik_match = cik_pattern.search(content)
            if name_match and cik_match:
                company_name = name_match.group(1).strip()
                cik = cik_match.group(1).strip()
                records.append((filename, company_name, cik))

# Convert to DataFrame
df = pd.DataFrame(records, columns=['File Name', 'Company Name', 'CIK'])
print('Count: ' + str(count))
print(df)


Count: 3788
                     File Name                         Company Name  \
0     0000077360-94-000007.txt                          PENTAIR INC   
1     0000846657-94-000008.txt                 ELJER INDUSTRIES INC   
2     0000045012-94-000047.txt                       HALLIBURTON CO   
3     0000902595-94-000001.txt     INTERNATIONAL LEASE FINANCE CORP   
4     0000716039-94-000029.txt           UNION OIL CO OF CALIFORNIA   
...                        ...                                  ...   
3783  0000857402-94-000163.txt  SEARS CREDIT ACCOUNT MASTER TRUST I   
3784  0000950123-94-001594.txt                           BORDEN INC   
3785  0000799319-94-000022.txt         IDB COMMUNICATIONS GROUP INC   
3786  0000892569-94-000277.txt                    PETERS J M CO INC   
3787  0000893220-94-000428.txt                           CIGNA CORP   

             CIK  
0     0000077360  
1     0000846657  
2     0000045012  
3     0000714311  
4     0000100880  
...          ...  
37

In [83]:
df['CIK'] = df['CIK'].astype(int).astype(str)

In [84]:
df

,File Name,Company Name,CIK
0,0000077360-94-000007.txt,PENTAIR INC,77360
1,0000846657-94-000008.txt,ELJER INDUSTRIES INC,846657
2,0000045012-94-000047.txt,HALLIBURTON CO,45012
3,0000902595-94-000001.txt,INTERNATIONAL LEASE FINANCE CORP,714311
4,0000716039-94-000029.txt,UNION OIL CO OF CALIFORNIA,100880
...,...,...,...
3783,0000857402-94-000163.txt,SEARS CREDIT ACCOUNT MASTER TRUST I,869391
3784,0000950123-94-001594.txt,BORDEN INC,13239
3785,0000799319-94-000022.txt,IDB COMMUNICATIONS GROUP INC,799319
3786,0000892569-94-000277.txt,PETERS J M CO INC,813945


In [85]:
gvkey_df = pd.read_csv('gvkey.csv', encoding='latin1')

# Convert dates to datetime
gvkey_df['sec_start_date'] = pd.to_datetime(gvkey_df['sec_start_date'], errors='coerce')
gvkey_df['sec_end_date'] = pd.to_datetime(gvkey_df['sec_end_date'], errors='coerce')

# Define the 1994 range
start_1994 = pd.to_datetime('1994-01-01')
end_1994 = pd.to_datetime('1994-12-31')

# Keep only rows where the mapping was valid at any time during 1994
gvkey_1994 = gvkey_df[
    (gvkey_df['sec_start_date'] <= end_1994) &
    (gvkey_df['sec_end_date'] >= start_1994)
]

# Convert CIK to int in gvkey_df as well
df['CIK'] = df['CIK'].astype(str).str.strip().astype(int)
gvkey_df['cik'] = gvkey_df['cik'].astype(str).str.strip().astype(int)

# Perform the merge on 'CIK' and 'cik', keeping only the 'gvkey' column from gvkey_df
merged_df = df.merge(gvkey_df[['cik', 'gvkey']], left_on='CIK', right_on='cik', how='left')
merged_df.drop(columns=['cik'], inplace=True)

# Now merged_df contains: File Name, Company Name, CIK, gvkey
print(merged_df.head())

                  File Name Company Name    CIK   gvkey
0  0000077360-94-000007.txt  PENTAIR INC  77360  8463.0
1  0000077360-94-000007.txt  PENTAIR INC  77360  8463.0
2  0000077360-94-000007.txt  PENTAIR INC  77360  8463.0
3  0000077360-94-000007.txt  PENTAIR INC  77360  8463.0
4  0000077360-94-000007.txt  PENTAIR INC  77360  8463.0


In [86]:
merged_df = merged_df.drop_duplicates()
merged_df = merged_df.dropna()

In [87]:
merged_df

,File Name,Company Name,CIK,gvkey
0,0000077360-94-000007.txt,PENTAIR INC,77360,8463.0
5,0000846657-94-000008.txt,ELJER INDUSTRIES INC,846657,15233.0
9,0000045012-94-000047.txt,HALLIBURTON CO,45012,5439.0
13,0000902595-94-000001.txt,INTERNATIONAL LEASE FINANCE CORP,714311,6089.0
16,0000716039-94-000029.txt,UNION OIL CO OF CALIFORNIA,100880,15251.0
...,...,...,...,...
13632,0000038009-94-000028.txt,FORD MOTOR CREDIT CO,38009,4840.0
13637,0000950123-94-001594.txt,BORDEN INC,13239,2316.0
13642,0000799319-94-000022.txt,IDB COMMUNICATIONS GROUP INC,799319,12628.0
13646,0000892569-94-000277.txt,PETERS J M CO INC,813945,13870.0


In [88]:
cusip_df = pd.read_csv('cusip.csv', encoding='latin1')

merged_df['CIK'] = merged_df['CIK'].astype(int)
cusip_df['CIK'] = cusip_df['CIK'].astype(int)

# Deduplicate cusip_df so there's only one row per CIK
cusip_unique = cusip_df.drop_duplicates(subset=['CIK'])

# Merge with your 3,820-row merged_df
merged_df = merged_df.merge(cusip_unique[['CIK', 'CUSIP_FULL', 'CUSIP']], on='CIK', how='left')
merged_df

,File Name,Company Name,CIK,gvkey,CUSIP_FULL,CUSIP
0,0000077360-94-000007.txt,PENTAIR INC,77360,8463.0,709631105,70963110
1,0000846657-94-000008.txt,ELJER INDUSTRIES INC,846657,15233.0,287161103,28716110
2,0000045012-94-000047.txt,HALLIBURTON CO,45012,5439.0,406216101,40621610
3,0000902595-94-000001.txt,INTERNATIONAL LEASE FINANCE CORP,714311,6089.0,NaN,NaN
4,0000716039-94-000029.txt,UNION OIL CO OF CALIFORNIA,100880,15251.0,NaN,NaN
...,...,...,...,...,...,...
3815,0000038009-94-000028.txt,FORD MOTOR CREDIT CO,38009,4840.0,NaN,NaN
3816,0000950123-94-001594.txt,BORDEN INC,13239,2316.0,099599102,09959910
3817,0000799319-94-000022.txt,IDB COMMUNICATIONS GROUP INC,799319,12628.0,449355106,44935510
3818,0000892569-94-000277.txt,PETERS J M CO INC,813945,13870.0,14040M104,14040M10


In [89]:
merged_df = merged_df.drop_duplicates()
merged_df = merged_df.dropna()
merged_df


,File Name,Company Name,CIK,gvkey,CUSIP_FULL,CUSIP
0,0000077360-94-000007.txt,PENTAIR INC,77360,8463.0,709631105,70963110
1,0000846657-94-000008.txt,ELJER INDUSTRIES INC,846657,15233.0,287161103,28716110
2,0000045012-94-000047.txt,HALLIBURTON CO,45012,5439.0,406216101,40621610
7,0000859119-94-000036.txt,ILLINOIS CENTRAL CORP,859119,23065.0,451841100,45184110
8,0000950103-94-003539.txt,FREEPORT MCMORAN COPPER & GOLD INC,831259,14590.0,35671D501,35671D50
...,...,...,...,...,...,...
3814,0000716039-94-000030.txt,UNOCAL CORP/DE,716039,11038.0,915289102,91528910
3816,0000950123-94-001594.txt,BORDEN INC,13239,2316.0,099599102,09959910
3817,0000799319-94-000022.txt,IDB COMMUNICATIONS GROUP INC,799319,12628.0,449355106,44935510
3818,0000892569-94-000277.txt,PETERS J M CO INC,813945,13870.0,14040M104,14040M10


zsh:1: command not found: pip


In [103]:
import os
import re
import pandas as pd
import spacy

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 2_000_000

# Folder with 8-K .txt files
data_folder = './data'  # <- change this to your actual path

records = []

count = 0

# Process each file
for filename in os.listdir(data_folder):
    if filename.endswith('.txt'):
        filepath = os.path.join(data_folder, filename)

        with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()

        text_cleaned = text.lower().replace('\r', ' ').replace('\n', ' ')
        if len(text_cleaned) > 2_000_000:
            print(f"⚠️ Skipping large file: {filename} ({len(text_cleaned)} chars)")
            continue
        doc = nlp(text_cleaned)

        # Extract sentences with the word "guidance"
        guidance_sentences = [sent.text for sent in doc.sents if 'guidance' in sent.text]

        # Naive paragraph split
        paragraphs = text.split('\n\n')
        guidance_paragraphs = [p.strip() for p in paragraphs if 'guidance' in p.lower()]

        if guidance_sentences or guidance_paragraphs:
            records.append({
                'File Name': filename,
                'Num Sentences': len(guidance_sentences),
                'Num Paragraphs': len(guidance_paragraphs),
                'Sentences': guidance_sentences,
                'Paragraphs': guidance_paragraphs
            })
        count += 1 
        print("Done with file number: " + str(count))

# Create and save DataFrame
guidance_df = pd.DataFrame(records)
guidance_df.to_csv('guidance_extracted.csv', index=False)

print("✅ Done. Extracted guidance content saved to 'guidance_extracted.csv'")


Done with file number: 1
Done with file number: 2
Done with file number: 3
Done with file number: 4
Done with file number: 5
Done with file number: 6
Done with file number: 7
Done with file number: 8
Done with file number: 9
Done with file number: 10
Done with file number: 11
Done with file number: 12
Done with file number: 13
Done with file number: 14
Done with file number: 15
Done with file number: 16
Done with file number: 17
Done with file number: 18
Done with file number: 19
Done with file number: 20
Done with file number: 21
Done with file number: 22
Done with file number: 23
Done with file number: 24
Done with file number: 25
Done with file number: 26
Done with file number: 27
Done with file number: 28
Done with file number: 29
Done with file number: 30
Done with file number: 31
Done with file number: 32
Done with file number: 33
Done with file number: 34
Done with file number: 35
Done with file number: 36
Done with file number: 37
Done with file number: 38
Done with file number